# 환경 변수 로드

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# 5-1

In [2]:
from langchain_community.tools import TavilySearchResults

# 검색할 쿼리 
query = "2024년 4분기 테슬라의 주가 전망은?"

# 최대 2개의 결과를 가져올 수 있도록 설정
web_search = TavilySearchResults(max_results=2)

# 검색 쿼리를 이용해 결과를 호출 (invoke)
search_results = web_search.invoke(query)

# 검색 결과를 반복문을 통해 출력
for result in search_results:
    print(result)
    print("-" * 100)

{'url': 'https://blog.naver.com/PostView.naver?blogId=economy_gull&logNo=223322349603', 'content': '2024년 테슬라의 주가 전망은 어떻게 될까요? 1년 사이에 2배가 되었지만 최고 전 고점은 21년 4월 11일에 세운 411달러입니다. ... 전기자동차 업체 byd는 2023년도 4분기 50만 대를 판매하여 테슬라의 판매량을 넘었습니다.'}
----------------------------------------------------------------------------------------------------
{'url': 'https://stockscan.io/ko/stocks/TSLA/forecast', 'content': '테슬라의 주가(tsla) 전망은 다음 12개월 동안 일반적으로 긍정적인데요. 애널리스트들의 평균 예상 주가는 311.93달러로, 현재 주가인 255.99달러 대비 +21.85% 증가을 나타내고 있습니다. ... 2024: $324.42: $298.92: $324.94 +26.73%: 11월, 2024: $352.69: $329.70: $352.98 +37.77%: 12월, 2024 ...'}
----------------------------------------------------------------------------------------------------


# 5-2

In [3]:
from langchain_openai import ChatOpenAI

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search])

# 사용자의 질문 작성 
query = "2024년 4분기 테슬라의 주가 전망은?"

# LLM에 질문을 전달하고 도구 호출 결과를 받음 
tools_output = llm_with_tools.invoke(query)

# 도구 호출 결과를 출력
print(tools_output)
print("-" * 100)

# 도구 호출에 대한 세부 정보를 출력 
print(tools_output.tool_calls)
print("-" * 100)

content='' additional_kwargs={'tool_calls': [{'id': 'call_WLrQd9P3ItStXtdsDOlYS6IT', 'function': {'arguments': '{"query":"2024년 4분기 테슬라 주가 전망"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 95, 'total_tokens': 125, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-3384910a-a40e-4883-9ada-f650b09570b3-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 4분기 테슬라 주가 전망'}, 'id': 'call_WLrQd9P3ItStXtdsDOlYS6IT', 'type': 'tool_call'}] usage_metadata={'input_tokens': 95, 'output_tokens': 30, 'total_tokens': 125}
----------------------------------------------------------------------------------------------------
[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 4분기 테슬라 주가 전망'}, 'id': 'call_WLrQd9P3ItStXtds

# 5-3

In [4]:
from langchain_core.messages import HumanMessage

# 질문을 메시지 목록에 추가
query = "2024년 4분기 테슬라의 주가 전망은?"
messages = [HumanMessage(query)]

# 도구 호출 메시지 추가
messages.append(tools_output)

# 도구 출력값을 메시지에 추가 
for tool_call in tools_output.tool_calls:
    # 호출할 도구 선택
    selected_tool = {"tavily_search_results_json": web_search}[tool_call["name"].lower()]
    
    # 도구 호출을 실행 
    tool_msg = selected_tool.invoke(tool_call)

    # 결과 메시지를 추가
    messages.append(tool_msg)

# 최종 모델 호출
final_response = llm_with_tools.invoke(messages)

# 최종 응답 출력
print(messages)
print("-" * 100)

print(final_response.content)
print("-" * 100)

print(final_response.response_metadata)
print("-" * 100)

[HumanMessage(content='2024년 4분기 테슬라의 주가 전망은?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WLrQd9P3ItStXtdsDOlYS6IT', 'function': {'arguments': '{"query":"2024년 4분기 테슬라 주가 전망"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 95, 'total_tokens': 125, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3384910a-a40e-4883-9ada-f650b09570b3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 4분기 테슬라 주가 전망'}, 'id': 'call_WLrQd9P3ItStXtdsDOlYS6IT', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 30, 'total_tokens': 125}), ToolMessage(content='[{"url": "https://www.chosun.com/economy/tech_it/2024/01/26/JEB5V42CHBEC3NMDBQWD

# 5-4

In [5]:
import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
response = search_chain.invoke("다음 분기 테슬라의 주가 전망은?")

# 응답 출력 
print(response.content)

다음 분기 테슬라(Tesla)의 주가 전망에 대한 분석은 다음과 같습니다:

1. **주요 분석가 의견**:
   - 33명의 분석가에 따르면, 테슬라 주식의 12개월 평균 목표가는 **$207.51**입니다. 이 중 낮은 예상가는 **$24.86**이며, 높은 예상가는 **$310**입니다. 현재 주가가 **$250**인 점을 고려할 때, 평균 목표가는 약 **17%**의 하락을 예측하고 있습니다. [자세한 내용 보기](https://stockanalysis.com/stocks/tsla/forecast/)

2. **추가 분석**:
   - 38명의 분석가의 분석에 따르면, 테슬라의 12개월 평균 가격 목표는 **$209.60**입니다. 이 또한 현재 가격인 **$254.27**에서 약 **17.57%**의 하락을 예상하고 있습니다. [자세한 내용 보기](https://www.marketbeat.com/stocks/NASDAQ/TSLA/forecast/)

종합적으로, 현재 분석가들은 테슬라의 주가가 다음 분기에는 하락할 것으로 보며, 평균 목표가는 현재 주가보다 낮게 설정되어 있습니다.


# 5-5

In [6]:
# 체인 실행
response = search_chain.invoke("당신의 이름은 무엇인가요?")

# 응답 출력 
print(response.content)

저는 AI 어시스턴트입니다. 특별한 이름은 없지만, 도와드릴 준비가 되어 있습니다! 무엇을 도와드릴까요?


# 5-6

In [7]:
# 사용자의 질문 작성 
query = "당신의 이름은 무엇인가요?"

# LLM에 질문을 전달하고 도구 호출 결과를 받음 
tools_output = llm_with_tools.invoke(query)

# 도구 호출 결과를 출력
print(tools_output)
print("-" * 100)

# 도구 호출에 대한 세부 정보를 출력 
print(tools_output.tool_calls)
print("-" * 100)

content='저는 AI 언어 모델이기 때문에 특정한 이름은 없습니다. 하지만 "Assistant"나 "AI"라고 불러주시면 됩니다. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 88, 'total_tokens': 126, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e9627b5346', 'finish_reason': 'stop', 'logprobs': None} id='run-4bcbccce-fb83-426b-be2f-ba7e15e4d56f-0' usage_metadata={'input_tokens': 88, 'output_tokens': 38, 'total_tokens': 126}
----------------------------------------------------------------------------------------------------
[]
----------------------------------------------------------------------------------------------------


# 5-7

In [8]:
from langchain_core.tools import tool
import yfinance as yf

@tool
def get_stock_price(ticker: str) -> float:
    """주어진 주식 티커의 현재 주가를 조회합니다.

    Args:
        ticker: 주식 티커 심볼 (예: 'AAPL'은 Apple을 의미).

    Returns:
        현재 주가를 float 형식으로 반환합니다.
    """
    # Yahoo Finance를 사용하여 주식 데이터 가져오기
    stock = yf.Ticker(ticker)
    # 현재 주식 가격 가져오기 (최근 거래일의 종가)
    stock_price = stock.history(period='1d')['Close'].iloc[0]
    return stock_price

# 도구 속성 확인
print("자료형: ")
print(type(get_stock_price))
print("-"*100)

print("name: ")
print(get_stock_price.name)
print("-"*100)

print("description: ")
print(get_stock_price.description)
print("-"*100)

print("args: ")
print(get_stock_price.args)
print("-"*100)

# 함수의 인자로 티커 전달하여 실행
current_price = get_stock_price.invoke({"ticker":"AAPL"})
print(f"AAPL의 현재 주가: ${current_price:.2f}")
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
get_stock_price
----------------------------------------------------------------------------------------------------
description: 
주어진 주식 티커의 현재 주가를 조회합니다.

    Args:
        ticker: 주식 티커 심볼 (예: 'AAPL'은 Apple을 의미).

    Returns:
        현재 주가를 float 형식으로 반환합니다.
----------------------------------------------------------------------------------------------------
args: 
{'ticker': {'title': 'Ticker', 'type': 'string'}}
----------------------------------------------------------------------------------------------------
AAPL의 현재 주가: $227.52
----------------------------------------------------------------------------------------------------


# 5-8

In [9]:
# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[get_stock_price])

# 사용자의 질문 작성 
query = "테슬라의 주가는 얼마인가요?"

# LLM에 질문을 전달하고 도구 호출 결과를 받음 
tools_output = llm_with_tools.invoke(query)

# 도구 호출 결과를 출력
print(tools_output)
print("-" * 100)

# 도구 호출에 대한 세부 정보를 출력 (스키마에 대한 결과)
print(tools_output.tool_calls)
print("-" * 100)

# 도구를 실행하여 출력값을 생성
tool_msg = get_stock_price.batch(tools_output.tool_calls)

# 출력각을 표시 
print(tool_msg)
print("-" * 100)

content='' additional_kwargs={'tool_calls': [{'id': 'call_CypJ2xJTkQLPVFyxN7rspuE8', 'function': {'arguments': '{"ticker":"TSLA"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 98, 'total_tokens': 114, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0e47c3e9-40b1-4242-ae4c-468f58490715-0' tool_calls=[{'name': 'get_stock_price', 'args': {'ticker': 'TSLA'}, 'id': 'call_CypJ2xJTkQLPVFyxN7rspuE8', 'type': 'tool_call'}] usage_metadata={'input_tokens': 98, 'output_tokens': 16, 'total_tokens': 114}
----------------------------------------------------------------------------------------------------
[{'name': 'get_stock_price', 'args': {'ticker': 'TSLA'}, 'id': 'call_CypJ2xJTkQLPVFyxN7rspuE8', 'type': 'tool_call'}]
------------------------------------------

# 5-9

In [10]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[get_stock_price])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def stock_price_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = get_stock_price.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)


# 체인 실행
response = stock_price_chain.invoke("테슬라의 주가는 얼마인가요?")

# 응답 출력 
print(response.content)

테슬라(TSLA)의 현재 주가는 약 254.22 달러입니다.


# 5-10

In [11]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from textwrap import dedent
import yfinance as yf
from datetime import datetime, timedelta

# 입력 스키마 정의
class StockPriceInput(BaseModel):
    """주식 가격 조회 입력 스키마."""

    ticker: str = Field(description="주식 티커 심볼 (예: 'AAPL'은 Apple을 의미)")
    date: str = Field(description=dedent("""\
        조회할 날짜 (YYYY-MM-DD 형식). 
        비워두면 최근 거래일 사용"""), default="")

def get_stock_price_with_date(ticker: str, date: str) -> tuple:
    """주어진 주식 티커와 거래일의 주가를 조회합니다."""

    stock = yf.Ticker(ticker)
    
    # 사용자가 지정한 날짜로 데이터 조회
    end_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = end_date - timedelta(days=5)  # 5일 전부터 조회 (주말/공휴일 고려)
    stock_data = stock.history(start=start_date, end=end_date + timedelta(days=1))
    
    if stock_data.empty:
        return None, None  # 데이터가 없는 경우
    
    # 가장 최근 데이터의 날짜와 종가 반환
    last_trading_date = stock_data.index[-1].strftime('%Y-%m-%d')
    stock_price = stock_data['Close'].iloc[-1]
    
    return stock_price, last_trading_date


# StructuredTool로 도구 생성
stock_price_date_tool = StructuredTool.from_function(
    func=get_stock_price_with_date,
    name="GetStockPriceWithDate",
    description="주어진 주식 티커와 거래일의 주가를 조회합니다.",
    args_schema=StockPriceInput,
    return_direct=True
)

# 도구 사용 예시
result = stock_price_date_tool.invoke({"ticker": "TSLA", "date": "2023-12-25"})
if result[0] is not None:
    print(f"주가: ${result[0]:.2f}, 거래일: {result[1]}")
else:
    print("해당 날짜의 주가 정보를 찾을 수 없습니다.")

print(f"도구 이름: {stock_price_date_tool.name}")
print(f"도구 설명: {stock_price_date_tool.description}")
print(f"도구 인자: {stock_price_date_tool.args}")

주가: $252.54, 거래일: 2023-12-22
도구 이름: GetStockPriceWithDate
도구 설명: 주어진 주식 티커와 거래일의 주가를 조회합니다.
도구 인자: {'ticker': {'description': "주식 티커 심볼 (예: 'AAPL'은 Apple을 의미)", 'title': 'Ticker', 'type': 'string'}, 'date': {'default': '', 'description': '조회할 날짜 (YYYY-MM-DD 형식). \n비워두면 최근 거래일 사용', 'title': 'Date', 'type': 'string'}}


# 5-11

In [12]:
# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools([stock_price_date_tool])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 체인 정의
@chain
def stock_price_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = stock_price_date_tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = stock_price_chain.invoke("2023년 크리스마스 시즌 테슬라 주가는 얼마였나요?")

# 응답 출력
print(response.content)

2023년 크리스마스 시즌 동안 테슬라(주식 티커: TSLA)의 주가는 다음과 같았습니다:

- 2023년 12월 22일: $252.54
- 2023년 12월 24일: $252.54
- 2023년 12월 25일: 주식 시장이 휴장하여 거래가 없었습니다.

따라서 2023년 크리스마스 당일에는 주가가 기록되지 않았습니다.


# 5-12

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4o")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search, stock_price_date_tool])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def stock_analysis_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        try:
            if tool_call["name"] == "tavily_search_results_json":
                tool_result = web_search.invoke(tool_call)
            elif tool_call["name"] == "GetStockPriceWithDate":
                tool_result = stock_price_date_tool.invoke(tool_call)
            else:
                print(f"Unknown tool called: {tool_call['name']}")
                continue

            # tool message 출력
            print("Tool Name: ", tool_call["name"])
            print(tool_result)
            print("-"*100)

            tool_msgs.append(tool_result)
        except Exception as e:
            print(f"Error processing tool call {tool_call['name']}: {e}")
    
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
query = "테슬라의 어제 주가는 전기차 시장의 최근 상황을 반영하고 있나요?"
response = stock_analysis_chain.invoke(query)

# 응답 출력
print("최종 답변: ")
print(response.content)

Tool Name:  GetStockPriceWithDate
content='[254.22000122070312, "2024-09-26"]' name='GetStockPriceWithDate' tool_call_id='call_VIox6kvFLPjghlswwZOQ7jJg'
----------------------------------------------------------------------------------------------------
Tool Name:  tavily_search_results_json
content='[{"url": "https://www.experianplc.com/newsroom/press-releases/2024/new-experian-report-shows-a-shifting-electric-vehicle-market-in-", "content": "Schaumburg, Ill., September 26, 2024 — With a wider range of electric vehicle (EV) models being introduced, the EV landscape is beginning to shift. According to Experian\'s Automotive Consumer Trends Report: Q2 2024, non-luxury EVs are slowly making up more of the market, growing from 22.7% last year to 26.6% in Q2 2024.Meanwhile, the percentage of exotic and luxury models declined from 77. ..."}, {"url": "https://www.iea.org/reports/global-ev-outlook-2024", "content": "About this report. The Global EV Outlook is an annual publication that identi

# 5-13

In [14]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI

# Pydantic 모델 정의
class StockAnalysis(BaseModel):
    """주식 분석 결과"""
    stock_symbol: str = Field(description="분석 대상 주식의 심볼")
    current_price: float = Field(description="현재 주식 가격")
    price_date: str = Field(description="주가 정보 날짜")
    market_trend: str = Field(description="전반적인 시장 동향")
    key_factors: List[str] = Field(description="주가에 영향을 미치는 주요 요인들")
    analysis: str = Field(description="주가와 시장 상황에 대한 분석")

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 구조화된 출력을 위한 모델 생성
structured_llm = llm.with_structured_output(StockAnalysis)

# 모델 실행
result = structured_llm.invoke("테슬라(TSLA)의 현재 주가와 시장 동향을 분석해주세요.")

# 결과 출력
print(f"주식 심볼: {result.stock_symbol}")
print(f"현재 가격: ${result.current_price}")
print(f"시장 동향: {result.market_trend}")
print("주요 요인:")
for factor in result.key_factors:
    print(f"- {factor}")

print(f"분석: {result.analysis}")

주식 심볼: TSLA
현재 가격: $240.0
시장 동향: 상승세
주요 요인:
- 전기차 수요 증가
- 새로운 모델 출시 예고
- 배터리 기술 발전
- 경쟁사와의 차별화된 기술
분석: 테슬라는 최근 전기차 수요 증가와 함께 긍정적인 시장 반응을 얻고 있습니다. 새로운 모델 출시 예고와 함께 배터리 기술의 발전이 주가 상승을 이끌고 있으며, 경쟁사 대비 차별화된 기술력 또한 주가에 긍정적인 영향을 미치고 있습니다.


# 5-14

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from pydantic import BaseModel, Field
from typing import List

# Pydantic 모델 정의
class StockAnalysis(BaseModel):
    stock_symbol: str = Field(description="분석 대상 주식의 심볼")
    current_price: float = Field(description="현재 주식 가격")
    price_date: str = Field(description="주가 정보 날짜")
    market_trend: str = Field(description="전반적인 시장 동향")
    key_factors: List[str] = Field(description="주가에 영향을 미치는 주요 요인들")
    analysis: str = Field(description="주가와 시장 상황에 대한 분석")

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4o-2024-08-06")

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[web_search, stock_price_date_tool])

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 금융 분석가입니다. 오늘 날짜는 {today}입니다. "
               f"주어진 정보를 바탕으로 StockAnalysis 구조에 맞는 분석 결과를 생성하세요."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def stock_analysis_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        try:
            if tool_call["name"] == "tavily_search_results_json":
                tool_result = web_search.invoke(tool_call)
            elif tool_call["name"] == "GetStockPriceWithDate":
                tool_result = stock_price_date_tool.invoke(tool_call)
            else:
                print(f"Unknown tool called: {tool_call['name']}")
                continue

            # tool message 출력
            print("Tool Name: ", tool_call["name"])
            print(tool_result)
            print("-"*100)

            tool_msgs.append(tool_result)
        except Exception as e:
            print(f"Error processing tool call {tool_call['name']}: {e}")
    
    final_response = llm_chain.invoke(
        {**input_, "messages": [ai_msg, *tool_msgs]}, 
        config=config
        )
    
    # 구조화된 출력 생성
    structured_llm = llm.with_structured_output(StockAnalysis)

    structured_prompt = f"""Based on the following information, provide a structured analysis: {final_response.content}"""
    structured_output = structured_llm.invoke(structured_prompt)
        
    return structured_output

# 체인 실행
query = "테슬라의 어제 주가는 전기차 시장의 최근 상황을 반영하고 있나요?"
response = stock_analysis_chain.invoke(query)

# 응답 출력
print("구조화된 분석 결과:")
print(f"주식 심볼: {response.stock_symbol}")
print(f"현재 가격: ${response.current_price}")
print(f"가격 날짜: {response.price_date}")
print(f"시장 동향: {response.market_trend}")
print("주요 요인:")
for factor in response.key_factors:
    print(f"- {factor}")
print(f"분석: {response.analysis}")

Tool Name:  GetStockPriceWithDate
content='[254.22000122070312, "2024-09-26"]' name='GetStockPriceWithDate' tool_call_id='call_JmzSSPqHilgePAYSdZgeK6g0'
----------------------------------------------------------------------------------------------------
Tool Name:  tavily_search_results_json
content='[{"url": "https://arstechnica.com/cars/2024/09/ev-sales-remain-healthy-despite-online-doom-and-gloom/", "content": "it\'s just vibes — EV sales remain healthy despite online doom and gloom Battery EVs are expected to account for more than 8 percent. Jonathan M. Gitlin - Sep 26, 2024 5:40 pm UTC"}, {"url": "https://www.nytimes.com/2024/09/05/climate/electric-vehicle-sales-projections.html", "content": "Sept. 5, 2024. The race to electrify America\'s cars is looking more like a slog these days. In the past few months, several automakers announced plans to pull back on introducing new electric ..."}]' name='tavily_search_results_json' tool_call_id='call_QTrpY79aw2hmcbrn7gNQ3Zri' artifact={'qu